In [1]:
from notebooks.__init__ import *
%load_ext autoreload
%autoreload 2

# Load a config

In [5]:
# method_overrides = ["+method/detector=sift",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=0.99",
#                     "+test.evaluation.estimator.inlier_thr=0.7"]
# method_overrides = ["+method/detector=superpoint",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=0.98",
#                     "+test.evaluation.estimator.inlier_thr=1.0"]
# method_overrides = ["+method/detector=r2d2",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=0.99",
#                     "+test.evaluation.estimator.inlier_thr=1.0"]
# method_overrides = ["+method/detector=keynet",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=1.0",
#                     "+test.evaluation.estimator.inlier_thr=1.2"]
# method_overrides = ["+method/end_to_end=disk", 
#                     "+test.evaluation.ratio_test_thr=0.98", 
#                     "+test.evaluation.estimator.inlier_thr=0.9"]
# method_overrides = ["+method/detector=rekd",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=0.98",
#                     "+test.evaluation.estimator.inlier_thr=1.4"]
# method_overrides = ["+method/detector=shi_tomasi",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=0.99",
#                     "+test.evaluation.estimator.inlier_thr=0.7"]
# method_overrides = ["+method/detector=ness_st/reference",
#                     "+method/descriptor=disk",
#                     "+test.evaluation.ratio_test_thr=0.98",
#                     "+test.evaluation.estimator.inlier_thr=0.7"]
method_overrides = ["+method/detector=boness_st/reference",
                    "+method/descriptor=disk", 
                    "+test.evaluation.ratio_test_thr=0.99", 
                    "+test.evaluation.estimator.inlier_thr=0.7"]

# method_overrides = ["+method/detector=superpoint",
#                     "+method/descriptor=hardnet",
#                     "+test.evaluation.ratio_test_thr=0.93",
#                     "+test.evaluation.estimator.inlier_thr=2.8"]
# method_overrides = ["+method/detector=rekd",
#                     "+method/descriptor=hardnet",
#                     "+test.evaluation.ratio_test_thr=0.91",
#                     "+test.evaluation.estimator.inlier_thr=2.4"]
# method_overrides = ["+method/detector=boness_st/reference",
#                     "+method/descriptor=hardnet",
#                     "+test.evaluation.ratio_test_thr=0.92",
#                     "+test.evaluation.estimator.inlier_thr=2.4"]

dataset_overrides = ["+test/dataset=imc_pt", 
                     "+test/evaluation=two_view_geometry/fundamental_matrix"]
# dataset_overrides = ["+test/dataset=megadepth", 
#                      "+test/evaluation=two_view_geometry/fundamental_matrix"]
# dataset_overrides = ["+test/dataset=heb", 
#                      "+test/evaluation=two_view_geometry/homography"]
# dataset_overrides = ["+test/dataset=scannet",
                    #  "+test/evaluation=two_view_geometry/essential_matrix"]

In [18]:
with initialize_config_dir(config_dir='/home/konstantin/personal/Summertime/config', version_base='1.1'):
    cfg = compose(overrides=method_overrides + dataset_overrides + [        
        "test.dataset.pairs_loader.num_workers=0",
        "test.evaluation.parallelize=False",
        "+test/experiment=default",
        "+mock=True"
    ])

    OmegaConf.set_struct(cfg, False)
    
    cfg_wrapper = HydraConfigWrapper(cfg)

    cfg_wrapper.print()
    



Configuration file:
------------------------------------------------------------------------------------------
method:
  detector:
    name: boness_st
    checkpoint_url: file:///home/konstantin/personal/Summertime/weights/reference_model_epoch=15-avg_mAA=0.7263.ckpt
    model:
      input_channels: 3
      down:
      - 16
      - 32
      - 64
      - 64
      - 64
      up:
      - 64
      - 64
      - 64
      - 1
    base_detector:
      name: shi_tomasi
      sobel_size: 3
      window_size: 3
      window_cov: 2
      nms_size: 5
      localize: true
      score_thresh: 0.0
  descriptor:
    name: disk
    checkpoint_url: file:///home/konstantin/personal/Summertime/source/baselines/disk/disk/depth-save.pth
test:
  dataset:
    name: IMC-PT
    root_path: /mnt/sda/datasets/IMCPT
    csv_rel_path: test.csv
    pairs_csv_rel_path: test_pairs.csv
    loader:
      num_workers: 2
      num_samples: 8
    pairs_loader:
      batch_size: 2
      num_workers: 0
      num_samples: 4
 

# Instantiate a test task

In [35]:
from source.test_task import TestTask

feature_dir_path = Path("/home/konstantin/personal/features/notebooks")
test_task = TestTask.from_config(cfg, None, feature_dir_path)

if cfg.test.experiment.accelerator == 'gpu':
    test_task.to(torch.device('cuda'))

Loaded checkpoint for boness_st
Loaded checkpoint for disk


/home/konstantin/personal/Summertime/source/utils/method.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(input_model.get_weights(), map_location='cpu'

# Instantiate a dataset and get loaders

In [36]:
data_module = DataModule(test_dataset_cfg=cfg.test.dataset, 
                         evaluation_cfg=cfg.test.evaluation,
                         feature_dir_path=test_task.feature_dir_path)
data_module.setup(stage='test')

In [37]:
image_loader, feature_loader = data_module.test_dataloader()

# Compute features and calculate metrics

In [38]:
test_task.on_test_epoch_start()

In [39]:
with torch.no_grad():
    if test_task.detector is not None:
        test_task.detector.eval()

    if test_task.descriptor is not None:
        test_task.descriptor.eval()
        
    if test_task.end_to_end is not None:
        test_task.end_to_end.eval()

    for i, batch in enumerate(image_loader):
        if cfg.test.experiment.accelerator == 'gpu':
            for k in batch.keys():
                if isinstance(batch[k], torch.Tensor):
                    batch[k] = batch[k].to(torch.device('cuda'))
        
        test_task.test_step(batch, i, 0)

/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4373: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [40]:
results_list = []

for i, batch in enumerate(feature_loader):
    test_task.test_step(batch, i, 1)

results = np.concatenate(test_task.test_step_outputs, axis=0)

In [41]:
results.shape

(4, 7)

In [42]:
r_mAA = relative_pose_accuracy(results[:, 3].astype(np.float32), cfg.test.evaluation.max_r_err_thr).mean()
t_mAA = relative_pose_accuracy(results[:, 4].astype(np.float32), cfg.test.evaluation.max_t_err_thr).mean()

print("R mAA:", r_mAA)
print("t mAA:", t_mAA)

R mAA: 0.75
t mAA: 0.225


In [43]:
import shutil

shutil.rmtree(test_task.feature_dir_path)